<a href="https://colab.research.google.com/github/MrWou/Calcolo-Matrici-e-Distanze-Comuni-Emilia-Romagna/blob/main/GeoFrazioni_ER_Estrazione_Resiliente_Frazioni_Emilia_Romagna_da_OpenStreetMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📄 Documentazione del Progetto GeoFrazioni ER

Questo notebook Python è stato sviluppato per creare un dataset georeferenziato e preciso di tutte le frazioni dell'Emilia-Romagna, superando sfide comuni come l'omonimia dei nomi di località e i problemi di connettività con i server API.

### **Obiettivi Principali:**

*   **Geocoding:** Determinare i perimetri geografici esatti di ogni comune utilizzando l'API Nominatim di OpenStreetMap.
*   **Estrazione Dati:** Scaricare le informazioni relative alle frazioni (nodi `place=village` e `place=suburb`) tramite l'API Overpass, assicurandosi che rientrino nei perimetri comunali definiti.
*   **Resilienza:** Implementare strategie per gestire gli errori di rete (es. timeout, rate limiting) e garantire il completamento del processo attraverso la rotazione di server Overpass.
*   **Pulizia e Validazione:** Filtrare le frazioni che ricadono fuori dai confini regionali e comunali effettivi, e rimuovere duplicati per ottenere un dataset finale pulito e affidabile.

---

### **Sezione 1: Geocoding dei Comuni (Nominatim)**

**File di Input:** `Codice ISTAT Comuni e Provincie ER_update_07_2024.csv`
**File di Output:** `Comuni_ER_Georeferenziati.csv`

Questa fase iniziale si concentra sull'identificazione precisa dei confini geografici (Bounding Box) per ciascun comune dell'Emilia-Romagna. Viene utilizzata l'API **Nominatim** di OpenStreetMap per convertire i nomi dei comuni in coordinate geografiche e perimetri.

**Processo:**

1.  **Lettura Dati:** Carica un file CSV contenente l'elenco dei comuni e delle province dell'Emilia-Romagna.
2.  **Query Nominatim:** Per ogni comune, viene costruita una query specificando il nome del comune, la provincia, la regione (`Emilia-Romagna`) e il paese (`Italia`) per minimizzare le ambiguità (es. evitare `Lugo` in Spagna).
3.  **Estrazione Bounding Box:** Nominatim restituisce un bounding box (min_lat, max_lat, min_lon, max_lon) che delimita l'area del comune, insieme alle coordinate centrali (lat, lon) e l'ID OpenStreetMap.
4.  **Resilienza Nominatim:** Viene implementata una pausa di **1.1 secondi** tra una richiesta e l'altra per rispettare le politiche d'uso di Nominatim e prevenire il blocco dell'IP.
5.  **Salvataggio:** I dati georeferenziati (provincia, comune, osm_id, lat_centro, lon_centro, min_lat, max_lat, min_lon, max_lon) vengono salvati in un nuovo file CSV.

Questo passaggio è cruciale per creare una "gabbia geografica" che sarà utilizzata nella fase successiva per scaricare le frazioni, garantendo che i dati estratti appartengano effettivamente al comune desiderato.

### **Sezione 2: Estrazione Frazioni (Overpass Multi-Server)**

**File di Input:** `Comuni_ER_Georeferenziati.csv`
**File di Output:** `Frazioni_ER_Finali_Complete.csv`

Questa fase si occupa di estrarre le frazioni dai dati di OpenStreetMap utilizzando l'API Overpass, sfruttando i Bounding Box calcolati nello Step 1.

**Caratteristiche di Resilienza e Ottimizzazione:**

*   **Rotazione Server:** Viene mantenuta una lista di server Overpass (es. Germania, Kumi Systems, Russia). Se una richiesta fallisce o va in timeout su un server, lo script passa automaticamente al successivo, migliorando l'affidabilità.
*   **Gestione Rate Limit (429):** Se un server restituisce un errore `429 Too Many Requests`, lo script attende un tempo incrementale prima di riprovare sulla stessa risorsa, prevenendo il ban temporaneo.
*   **Gestione Errori Server (5xx) e Connessione:** In caso di errori del server (es. `504 Gateway Timeout`) o problemi di connessione, lo script tenta automaticamente la richiesta sul server successivo disponibile.
*   **Query Ottimizzata:** Le query Overpass sono formulate in modo efficiente (`node["place"="village"]` e `node["place"="suburb"]`) per ridurre il carico sulla CPU del server OSM e velocizzare l'estrazione.
*   **Esclusione Suburb Urbani:** Per i comuni capoluogo (es. Bologna, Modena, Parma), l'estrazione dei `place=suburb` viene disabilitata per evitare di includere aree urbane che non sono considerate frazioni nel contesto di questo progetto.

**Processo:**

1.  **Caricamento Bounding Box:** Legge il file `Comuni_ER_Georeferenziati.csv` con i perimetri di tutti i comuni.
2.  **Query Overpass:** Per ogni comune, costruisce una query Overpass per cercare nodi con tag `place=village` e, per i comuni non capoluogo, anche `place=suburb` all'interno del suo bounding box.
3.  **Estrazione Dati:** Analizza la risposta JSON dell'API, estraendo il nome della frazione, la tipologia (`village` o `suburb`), latitudine e longitudine.
4.  **Consolidamento:** Tutte le frazioni estratte vengono raccolte e arricchite con il nome della provincia di appartenenza.
5.  **Salvataggio Finale:** I dati consolidati vengono salvati in un nuovo file CSV, `Frazioni_ER_Finali_Complete.csv`.

In [ ]:
import requests
import pandas as pd
import time
import random

# --- CONFIGURAZIONE ---
INPUT_GEO_FILE = "Comuni_ER_Georeferenziati.csv"
OUTPUT_FINAL_FILE = "Frazioni_ER_Finali_Complete.csv"

CITIES_NO_SUBURBS = [
    "Bologna", "Modena", "Parma", "Reggio nell'Emilia", "Reggio Emilia",
    "Ferrara", "Piacenza", "Ravenna", "Rimini", "Forlì", "Cesena"
]

# LISTA SERVER DI ROTAZIONE
# Se uno fallisce, lo script prova il successivo.
OVERPASS_SERVERS = [
    "https://overpass-api.de/api/interpreter",       # Principale (Germania)
    "https://overpass.private.coffee/api/interpreter", # Kumi Systems (Molto affidabile)
    "https://maps.mail.ru/osm/tools/overpass/api/interpreter" # Russia (Spesso libero)
]

def get_fractions_in_bbox(bbox, city_name, use_suburbs=True, attempt=1, server_index=0):

    # Se abbiamo finito i server, ci arrendiamo
    if server_index >= len(OVERPASS_SERVERS):
        print(f" [!] Tutti i server hanno fallito per {city_name}.")
        return []

    current_server = OVERPASS_SERVERS[server_index]

    # Costruiamo la query
    parts = [f'node["place"="village"]({bbox["min_lat"]},{bbox["min_lon"]},{bbox["max_lat"]},{bbox["max_lon"]});']
    if use_suburbs:
        parts.append(f'node["place"="suburb"]({bbox["min_lat"]},{bbox["min_lon"]},{bbox["max_lat"]},{bbox["max_lon"]});')

    query = f"[out:json][timeout:90];\n(\n" + "\n".join(parts) + "\n);\nout body;"

    try:
        # Timeout connessione
        r = requests.get(current_server, params={'data': query}, timeout=100)

        # 1. RATE LIMIT (429) -> Aspetta e riprova sullo STESSO server
        if r.status_code == 429:
            wait_time = 10 * attempt
            print(f" (Rate Limit su {server_index+1} - attendo {wait_time}s)...", end=" ")
            time.sleep(wait_time)
            return get_fractions_in_bbox(bbox, city_name, use_suburbs, attempt + 1, server_index)

        # 2. SERVER ERROR (5xx) -> Prova il PROSSIMO server
        if r.status_code >= 500:
            print(f" [Err {r.status_code} su Server {server_index+1}] Cambio server...", end=" ")
            return get_fractions_in_bbox(bbox, city_name, use_suburbs, 1, server_index + 1)

        r.raise_for_status()
        data = r.json()

        fractions = []
        for el in data.get('elements', []):
            if 'tags' in el and 'name' in el['tags']:
                fractions.append({
                    'Comune': city_name, # Aggiungiamo qui per comodità
                    'Frazione': el['tags']['name'],
                    'Tipologia': el['tags'].get('place'),
                    'Latitudine': el['lat'],
                    'Longitudine': el['lon']
                })
        return fractions

    except (requests.exceptions.RequestException, Exception) as e:
        # 3. ERRORE CONNESSIONE (DNS, Timeout) -> Prova il PROSSIMO server
        print(f" [ErrConn su Server {server_index+1}] Cambio server...", end=" ")
        time.sleep(1)
        return get_fractions_in_bbox(bbox, city_name, use_suburbs, 1, server_index + 1)

# --- ESECUZIONE ---
try:
    df_geo = pd.read_csv(INPUT_GEO_FILE)
    all_results = []

    total = len(df_geo)
    print(f"Inizio estrazione Multi-Server per {total} comuni...")
    print("-" * 60)

    for index, row in df_geo.iterrows():
        comune = row["Comune"]
        provincia = row["Provincia"]
        use_suburbs = comune not in CITIES_NO_SUBURBS

        print(f"[{index+1}/{total}] {comune}...", end=" ", flush=True)

        frazioni = get_fractions_in_bbox(row, comune, use_suburbs=use_suburbs)

        if frazioni:
            print(f"-> {len(frazioni)}")
            # Arricchiamo i dati con la provincia
            for f in frazioni:
                f['Provincia'] = provincia
            all_results.extend(frazioni)
        else:
            print("-> 0")

        # Pausa standard tra comuni
        time.sleep(1.5)

    if all_results:
        df_final = pd.DataFrame(all_results)
        cols = ['Provincia', 'Comune', 'Frazione', 'Tipologia', 'Latitudine', 'Longitudine']
        df_final = df_final.reindex(columns=cols)
        df_final.to_csv(OUTPUT_FINAL_FILE, index=False)
        print("-" * 60)
        print(f"FINITO! Dati salvati in: {OUTPUT_FINAL_FILE}")
    else:
        print("Nessuna frazione trovata.")

except FileNotFoundError:
    print(f"Errore: File {INPUT_GEO_FILE} mancante. Esegui prima lo Script 1!")

Inizio estrazione Multi-Server per 330 comuni...
------------------------------------------------------------
[1/330] Agazzano... -> 1
[2/330] Alseno... -> 5
[3/330] Besenzone... -> 3
[4/330] Bettola... -> 7
[5/330] Bobbio...  (Rate Limit su 1 - attendo 10s)...  (Rate Limit su 1 - attendo 20s)... -> 5
[6/330] Borgonovo Val Tidone... -> 3
[7/330] Cadeo... -> 2
[8/330] Calendasco... -> 9
[9/330] Caorso... -> 5
[10/330] Carpaneto Piacentino... -> 3
[11/330] Castell'Arquato... -> 4
[12/330] Castel San Giovanni... -> 2
[13/330] Castelvetro Piacentino... -> 10
[14/330] Cerignale... -> 2
[15/330] Coli... -> 2
[16/330] Corte Brugnatella... -> 3
[17/330] Cortemaggiore... -> 5
[18/330] Farini... -> 3
[19/330] Ferriere... -> 2
[20/330] Fiorenzuola d'Arda... -> 3
[21/330] Gazzola... -> 6
[22/330] Gossolengo... -> 2
[23/330] Gragnano Trebbiense... -> 7
[24/330] Gropparello... -> 1
[25/330] Lugagnano Val d'Arda... -> 4
[26/330] Monticelli d'Ongina... -> 8
[27/330] Morfasso... -> 10
[28/330] Ottone..

### **Sezione 3: Filtro Frazioni su Base Limiti Regionali**

**File di Input:** `Frazioni_ER_Finali_Complete.csv`
**File di Output:** `Frazioni_ER_Geofiltrate.csv`

Questa sezione esegue un'importante fase di pulizia spaziale, rimuovendo le frazioni che, pur essendo state individuate in precedenza, ricadono *fuori* dai confini amministrativi effettivi della regione Emilia-Romagna.

**Processo:**

1.  **Caricamento Dati:** Il file `Frazioni_ER_Finali_Complete.csv` viene caricato in un DataFrame Pandas.
2.  **Conversione a GeoDataFrame:** I punti delle frazioni (latitudine, longitudine) vengono convertiti in oggetti geografici (GeoDataFrame) con il sistema di coordinate WGS84 (EPSG:4326), standard per i dati GPS.
3.  **Caricamento Confini Regionali:** Viene caricato un file Shapefile (`.shp`), che rappresenta la geometria precisa dei confini della regione Emilia-Romagna.
4.  **Spatial Join (`predicate="within"`):** Viene eseguita un'operazione di *spatial join* tra il GeoDataFrame delle frazioni e il GeoDataFrame dei confini regionali. L'argomento `predicate="within"` assicura che vengano mantenute solo le frazioni i cui punti rientrano geometricamente all'interno del poligono della regione.
5.  **Salvataggio:** Il dataset risultante, contenente solo le frazioni validate entro i confini regionali, viene salvato in `Frazioni_ER_Geofiltrate.csv`. Viene anche mostrato il numero di punti rimossi in questa fase.

In [ ]:
import pandas as pd
import geopandas as gpd

# 1. Carica il CSV delle frazioni che hai appena creato
input_csv = "Frazioni_ER_Finali_Complete.csv"
df = pd.read_csv(input_csv)

# 2. Converti il CSV in Geometria (punti)
# WGS84 è il sistema di default di OSM
gdf_frazioni = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Longitudine, df.Latitudine),
    crs="EPSG:4326"
)

# 3. Carica lo Shapefile dei confini regionali (quello che hai appena scaricato)
# Sostituisci 'NOME_FILE_SCARICATO.shp' con il nome reale del file estratto dallo zip
shapefile_path = "/content/V_REG_GPG_1.shp"
gdf_confini = gpd.read_file(shapefile_path)

# Assicuriamoci che anche i confini siano in WGS84 (dovrebbero già esserlo se hai scaricato quello giusto)
if gdf_confini.crs != "EPSG:4326":
    gdf_confini = gdf_confini.to_crs("EPSG:4326")

# 4. IL FILTRO: Spatial Join
# Mantiene solo le frazioni che sono fisicamente DENTRO l'area della regione
print("Sto filtrando i punti fuori regione...")
gdf_pulito = gpd.sjoin(gdf_frazioni, gdf_confini, how="inner", predicate="within")

# 5. Salva il risultato pulito
output_clean = "Frazioni_ER_Geofiltrate.csv"
cols_to_keep = ['Provincia', 'Comune', 'Frazione', 'Tipologia', 'Latitudine', 'Longitudine']

# Se nel dataframe pulito ci sono colonne doppie o sporche, teniamo solo quelle originali
gdf_pulito[cols_to_keep].to_csv(output_clean, index=False)

punti_rimossi = len(df) - len(gdf_pulito)
print(f"Fatto! Rimossi {punti_rimossi} punti che erano fuori confine.")
print(f"Dataset pulito salvato come: {output_clean}")

Sto filtrando i punti fuori regione...
Fatto! Rimossi 76 punti che erano fuori confine.
Dataset pulito salvato come: Frazioni_ER_Geofiltrate.csv


### **Sezione 4: Rimozione Duplicati e Assegnazione Comune per Corrispondenza Geometrica**

**File di Input:** `Frazioni_ER_Geofiltrate.csv`
**File di Output:** `Frazioni_ER_Definitive_GeoValidate.csv`

Questa fase affina ulteriormente il dataset, risolvendo problemi di errata attribuzione del comune e rimuovendo duplicati basandosi su una validazione geometrica più precisa a livello comunale.

**Problematica:** A volte, a causa di imprecisioni nelle API o nella formulazione delle query iniziali, una frazione può essere erroneamente associata a un comune adiacente (problema di omonimia) o un punto può essere duplicato.

**Soluzione:**

1.  **Caricamento Dati:** Il file `Frazioni_ER_Geofiltrate.csv` (già filtrato a livello regionale) viene caricato.
2.  **Conversione a GeoDataFrame:** Come nella fase precedente, le frazioni vengono convertite in un GeoDataFrame di punti.
3.  **Caricamento Confini Comunali:** Viene caricato un file Shapefile (`.shp`) che contiene i poligoni geometrici di tutti i comuni dell'Emilia-Romagna (es. `V_COM_GPG_1.shp`).
4.  **Spatial Join Avanzato:** Viene eseguito uno *spatial join* tra le frazioni e i confini comunali. Per ogni frazione, viene identificato il comune esatto (`NOME_C` dallo Shapefile comunale) in cui ricade geometricamente.
5.  **Assegnazione Comune Reale:** La colonna `Comune` del dataset delle frazioni viene aggiornata con il nome del comune geometricamente corretto (`Comune_Reale`).
6.  **Rimozione Duplicati:** Una volta che ogni frazione è correttamente associata al suo comune geometrico, vengono rimossi i duplicati basandosi sulla combinazione `Comune`, `Frazione`, `Latitudine` e `Longitudine`. Questo assicura che ogni punto frazione sia unico e associato al comune corretto.
7.  **Salvataggio:** Il dataset pulito e validato geometricamente viene salvato in `Frazioni_ER_Definitive_GeoValidate.csv`.

In [7]:
import pandas as pd
import geopandas as gpd

# 1. Carica il CSV con le frazioni (quello che contiene ancora i duplicati/errori)
input_csv = "Frazioni_ER_Geofiltrate.csv"
df = pd.read_csv(input_csv)

# 2. Crea il GeoDataFrame dai punti GPS
gdf_punti = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Longitudine, df.Latitudine),
    crs="EPSG:4326"
)

# 3. Carica il TUO Shapefile Regionale
# Assicurati che 'shapefile_path' punti al file .shp che hai caricato (es. V_REG_GPG_1.shp o simile)
shapefile_path = "V_COM_GPG_1.shp"  # <--- SOSTITUISCI CON IL NOME ESATTO DEL TUO FILE .shp
gdf_comuni = gpd.read_file(shapefile_path)

# Convertiamo in coordinate GPS se necessario
if gdf_comuni.crs != "EPSG:4326":
    gdf_comuni = gdf_comuni.to_crs("EPSG:4326")

# 4. SPATIAL JOIN
# "Dimmi in quale poligono cade ogni punto"
print("Eseguo l'incrocio geografico (Spatial Join)...")
gdf_corretto = gpd.sjoin(gdf_punti, gdf_comuni, how="inner", predicate="within")

# 5. ASSEGNAZIONE DEL COMUNE CORRETTO
# Usiamo la colonna 'NOME_C' che abbiamo identificato nella tua lista
gdf_corretto['Comune_Reale'] = gdf_corretto['NOME_C']

# Selezioniamo le colonne finali
colonne_da_tenere = ['Provincia', 'Comune_Reale', 'Frazione', 'Tipologia', 'Latitudine', 'Longitudine']
df_finale = gdf_corretto[colonne_da_tenere].copy()
df_finale.rename(columns={'Comune_Reale': 'Comune'}, inplace=True)

# 6. PULIZIA FINALE (RIMOZIONE DUPLICATI)
# Ora che ogni punto ha il suo VERO comune, rimuoviamo i doppioni
n_prima = len(df_finale)
df_finale = df_finale.drop_duplicates(subset=['Comune', 'Frazione', 'Latitudine', 'Longitudine'])
n_dopo = len(df_finale)

print(f"Fatto! Sostituiti i comuni errati con quelli reali geometrici.")
print(f"Rimossi {n_prima - n_dopo} duplicati.")
print(df_finale.head())

# Salvataggio
df_finale.to_csv("Frazioni_ER_Definitive_GeoValidate.csv", index=False)

Eseguo l'incrocio geografico (Spatial Join)...
Fatto! Sostituiti i comuni errati con quelli reali geometrici.
Rimossi 1538 duplicati.
  Provincia              Comune              Frazione Tipologia  Latitudine  \
0  Piacenza            AGAZZANO              Agazzano   village   44.945661   
1  Piacenza              ALSENO                Alseno   village   44.896935   
2  Piacenza  FIORENZUOLA D`ARDA          Baselicaduce   village   44.944071   
3  Piacenza     CASTELL`ARQUATO           San Lorenzo    suburb   44.865730   
4  Piacenza              ALSENO  Castelnuovo Fogliani   village   44.875375   

   Longitudine  
0     9.518564  
1     9.965317  
2     9.940234  
3     9.897844  
4     9.973893  


### **Sezione 5: Pulizia Finale e Rimozione Duplicati Nomi Frazione**

**File di Input:** `Frazioni_ER_Definitive_GeoValidate.csv`
**File di Output:** `Frazioni_ER_Definitive_Final.csv`

L'ultimo passaggio del processo consiste in una pulizia finale per eliminare eventuali duplicati basati unicamente sulla combinazione di `Comune` e `Frazione`.

**Motivazione:** Dopo le fasi di validazione geometrica, potremmo ancora avere casi rari in cui due punti distinti, ma molto vicini, con lo stesso nome di frazione all'interno dello stesso comune, sono stati erroneamente mantenuti. Questa fase assicura che per ogni combinazione univoca di (Comune, Frazione) esista un solo punto nel dataset finale.

**Processo:**

1.  **Caricamento Dati:** Il file `Frazioni_ER_Definitive_GeoValidate.csv` viene caricato.
2.  **Rimozione Duplicati:** Viene applicata l'operazione `drop_duplicates` sul DataFrame, considerando le colonne `Comune` e `Frazione`. `keep='first'` garantisce che venga mantenuta la prima occorrenza trovata, scartando le successive.
3.  **Salvataggio:** Il dataset finale e completamente pulito viene salvato come `Frazioni_ER_Definitive_Final.csv`.

Questo file rappresenta il risultato finale del progetto, contenendo le frazioni dell'Emilia-Romagna georeferenziate, validate, e prive di duplicati o errori di attribuzione.

In [9]:
import pandas as pd

# 1. Carica il file validato geometricamente
file_path = "Frazioni_ER_Definitive_GeoValidate.csv"
df = pd.read_csv(file_path)

print(f"Righe iniziali: {len(df)}")

# 2. Rimuovi i duplicati basandoti SOLO su Comune e Nome Frazione
# (keep='first' tiene la prima occorrenza trovata e scarta le altre)
df_pulito = df.drop_duplicates(subset=['Comune', 'Frazione'], keep='first')

print(f"Righe finali: {len(df_pulito)}")
print(f"Rimossi {len(df) - len(df_pulito)} duplicati interni (es. i doppi Bizzuno, Musiano, ecc).")

# 3. Salva il file definitivo finale
output_file = "Frazioni_ER_Definitive_Final.csv"
df_pulito.to_csv(output_file, index=False)

print(f"Dataset perfetto salvato in: {output_file}")

Righe iniziali: 1608
Righe finali: 1605
Rimossi 3 duplicati interni (es. i doppi Bizzuno, Musiano, ecc).
Dataset perfetto salvato in: Frazioni_ER_Definitive_Final.csv


### **Sezione 6: Analisi del Dataset Finale**

Questa sezione offre un'analisi riassuntiva del dataset finale, fornendo statistiche sui nomi delle frazioni e verificando l'assenza di omonimie all'interno dello stesso comune.

**Processo:**

1.  **Caricamento Dataset:** Carica il file `Frazioni_ER_Definitive_Final.csv`.
2.  **Conteggio Frazioni:** Mostra il numero totale di frazioni nel dataset e il numero di nomi di frazione unici.
3.  **Top Nomi Comuni:** Visualizza i 20 nomi di frazione più frequenti nel dataset, utile per identificare omonimie a livello regionale.
4.  **Verifica Omonimie Interne al Comune:** Controlla se esistono frazioni con lo stesso nome all'interno dello *stesso* comune. Grazie alle fasi precedenti di pulizia e validazione geometrica, questo controllo dovrebbe idealmente non rilevare alcuna omonimia, confermando l'integrità del dataset.

Questa analisi finale serve a convalidare la qualità del dataset prodotto e a fornire una panoramica sulle sue caratteristiche.

In [11]:
import pandas as pd

# 1. Carica il file generato
file_path = "/content/Frazioni_ER_Definitive_Final.csv"
df = pd.read_csv(file_path)

# 2. Conta quante volte appare ogni nome di frazione
conteggio_nomi = df['Frazione'].value_counts()

print(f"Totale frazioni nel dataset: {len(df)}")
print(f"Numero di nomi di frazione unici: {len(conteggio_nomi)}")
print("\n--- Top 20 nomi di frazione più comuni ---")
print(conteggio_nomi.head(20))

# 3. Controllo avanzato: Nomi duplicati nello STESSO comune?
# (Questo non dovrebbe succedere se abbiamo rimosso i duplicati geometrici,
# a meno che un comune non abbia davvero due luoghi con lo stesso nome)
duplicati_interni = df[df.duplicated(subset=['Comune', 'Frazione'], keep=False)]

if not duplicati_interni.empty:
    print("\n--- ATTENZIONE: Frazioni con lo stesso nome nello STESSO comune ---")
    print(duplicati_interni.sort_values(by=['Comune', 'Frazione'])[['Comune', 'Frazione', 'Latitudine', 'Longitudine']])
else:
    print("\nOttimo: Non ci sono frazioni omonime all'interno dello stesso comune.")

Totale frazioni nel dataset: 1605
Numero di nomi di frazione unici: 1522

--- Top 20 nomi di frazione più comuni ---
Frazione
Cella             5
San Martino       5
Villanova         4
San Michele       3
San Prospero      3
San Biagio        3
Albareto          3
San Vito          3
Crocetta          3
Sant'Andrea       3
Villafranca       2
Ospedaletto       2
Santa Giustina    2
Borra             2
Cognento          2
Medelana          2
Osteria Nuova     2
Gavello           2
Frescarolo        2
Guarda            2
Name: count, dtype: int64

Ottimo: Non ci sono frazioni omonime all'interno dello stesso comune.


In [15]:
# 1. Carica il file CSV (Sostituisci con il nome del tuo file)
nome_file = "/content/Frazioni_ER_Definitive_Final.csv"
df = pd.read_csv(nome_file)

# 2. Filtra le righe dove la colonna 'Frazione' è uguale a 'Cella'
righe_cella = df[df['Frazione'] == 'San Prospero']

# 3. Mostra il risultato a video
print(f"Trovate {len(righe_cella)} righe con nome 'Cella':")
print(righe_cella)

Trovate 3 righe con nome 'Cella':
     Provincia        Comune      Frazione Tipologia  Latitudine  Longitudine
520     Modena  SAN PROSPERO  San Prospero   village   44.789671    11.022100
1043   Bologna         IMOLA  San Prospero   village   44.363340    11.775688
1222   Bologna   VALSAMOGGIA  San Prospero   village   44.364638    11.068234
